In [2]:
import sys
import lib.Buses as Buses
import gmaps
import config

gmaps.configure(api_key=config.free_maps_api_key)

In [3]:
# return a list of routes, by direction

##
## I THINK WE ARE STILL GETTING BACK BOTH DIRECTIONS FOR EACH DIRECTION FROM parse_route_xml
## NEED TO GROUND TRUTH THE POINTS - MAYBE FOR A SHORTER ROUTE?
##

routes=Buses.parse_route_xml(Buses.get_xml_data('nj', 'routes', route=119))


In [4]:
# this creates a new list of lists to store each direction

## CAN DEPRECATE AFTER UPDATING parse_routes_xml
##


# [
# [<direction1 Stop1>,<Point1>,<Point2>,<Stop2>,<Point3>..],
# [<direction2 Stop1>,<Point1>,<Point2>,<Stop2>,<Point3>..]
# ]

route_points_by_direction = []
for direction in routes:
    stops_and_points = [] 
    for path in direction.paths:
        for point in path.points:
            stops_and_points.append(point)     
    route_points_by_direction.append(stops_and_points) # appends to each list

    

In [9]:
##################################################################################################
# DIAGNOSTIC - why are we getting back both directions? what's wrong with parse_route_xml ??
# i checked - its not this notebook causing it
##################################################################################################
 
stops_points=route_points_by_direction[0]

for p in stops_points:
    if isinstance(p, Buses.Route.Stop):
            print p.st, float(p.lat), float(p.lon)
    else:
            print float(p.lat), float(p.lon)

JFK BLVD + 3RD ST 40.6486966667 -74.1376883333
40.6494281549 -74.1370296478
JFK BLVD + 4TH ST 40.6505166667 -74.1361783333
40.6511049885 -74.1356778145
40.6514631416 -74.1354632378
40.6520329268 -74.1353559494
40.6526515452 -74.1352057457
JFK BLVD + W 7TH STREET 40.65318 -74.13506
40.6536520959 -74.1346093889
40.6539776773 -74.1341373201
40.6543846519 -74.1336437936
40.6546288354 -74.1332146402
40.6548079028 -74.13257091
40.6550683636 -74.132313418
JFK BLVD + 8TH ST 40.65639 -74.1311
JFK BLVD + 10TH ST 40.65773 -74.12996
JFK BLVD + 12TH ST 40.65856 -74.12937
JFK BLVD + 14TH ST 40.66005 -74.12799
JFK BLVD + 16TH ST 40.66154 -74.12674
JFK BLVD + ANDREW ST 40.66276 -74.12575
JFK BLVD + 19TH ST 40.6639 -74.1247
JFK BLVD + 21ST ST 40.66507 -74.12369
JFK BLVD + 23RD ST 40.66647 -74.12251
JFK BLVD + 27TH ST 40.6697 -74.11978
JFK BLVD + 30TH ST 40.67167 -74.11815
JFK BLVD + 32ND ST 40.67322 -74.11686
JFK BLVD + 34TH ST 40.67568 -74.1148
40.6759530006 -74.1146171093
40.6762589447 -74.1144711974

In [5]:
# gmaps tutorial https://jupyter-gmaps.readthedocs.io/en/latest/tutorial.html#base-maps

In [6]:
# separate figures for route points and stop points (Alex original version) - direction inbound
route_points = []
stops_points = []
direction = 0

for path in routes[direction].paths:
    for point in path.points:
        if isinstance(point, Buses.Route.Stop):
            stops_points.append(point)
        else:
            route_points.append(point)
            
m = gmaps.Map()

m.add_layer(gmaps.symbol_layer([(float(p.lat), float(p.lon)) for p in route_points], fill_color='green', stroke_color='green', scale=2))
m.add_layer(gmaps.symbol_layer([(float(p.lat), float(p.lon)) for p in stops_points], fill_color='red', stroke_color='red', scale=5))
m

TWFwKGNvbmZpZ3VyYXRpb249eydhcGlfa2V5JzogdSdBSXphU3lCOFNIT0VhaW9qTlZ4QWRnMjQ2NlktdWpua2UwWWZlUkUnfSwgZGF0YV9ib3VuZHM9Wyg0MC42NTExMzAyNTA4NzE3LCAtNzTigKY=


In [7]:
# diagnostic map - stops only - direction inbound
direction = 0

map_points=route_points_by_direction[direction]

m = gmaps.Map()
# old version separately mapped route points and stop points
# m.add_layer(gmaps.symbol_layer([(float(p.lat), float(p.lon)) for p in route_points], fill_color='green', stroke_color='green', scale=2))
m.add_layer(gmaps.symbol_layer([(float(p.lat), float(p.lon)) for p in map_points], fill_color='red', stroke_color='red', scale=5))
m

TWFwKGNvbmZpZ3VyYXRpb249eydhcGlfa2V5JzogdSdBSXphU3lCOFNIT0VhaW9qTlZ4QWRnMjQ2NlktdWpua2UwWWZlUkUnfSwgZGF0YV9ib3VuZHM9Wyg0MC42NjA0NTY1NzgxMjc4OSwgLTfigKY=


In [8]:
# map - simple route (no waypoints) - direction inbound
direction = 0

# set up the directions-eligible API key
import config
gmaps.configure(api_key=config.paid_directions_api_key)

map_points=route_points_by_direction[direction]

# get length of the array and use that as the end point
# WHY IS IT STILL DOUBLED!!!! 
end_num = (len(map_points) - 1)/2  # ----> divide by two to fix


start = (float(map_points[0].lat), float(map_points[0].lon))
end = (float(map_points[end_num].lat), float(map_points[end_num].lon))

fig = gmaps.figure()
start2end = gmaps.directions_layer(start, end)
fig.add_layer(start2end)
fig



Figure(layout=FigureLayout(height=u'420px'))

In [9]:
# waypoint map


In [16]:
#  from python cookbook
#  Split an iterable p into a list of n sublists, repeatedly taking
#         the next element of p and adding it to the next sublist.  Example:
#         >>> strider('abcde', 3)
#         [['a', 'd'], ['b', 'e'], ['c']]
#         In other words, strider's result is equal to:
#             [list(p[i::n]) for i in xrange(n)]
#         if iterable p is a sequence supporting extended-slicing syntax.
#   

def strider(p, n):

    # First, prepare the result, a list of n separate lists
    result = [ [  ] for x in xrange(n) ]
    # Loop over the input, appending each item to one of
    # result's lists, in "round robin" fashion
    for i, item in enumerate(p):
        result[i % n].append(item)
    return result

In [17]:
# slice desired direction
direction = 0
map_points=route_points_by_direction[direction]
map_points

[<lib.Buses.Stop instance at 0x10422b5f0>,
 <lib.Buses.Point instance at 0x10422b680>,
 <lib.Buses.Point instance at 0x10422b710>,
 <lib.Buses.Point instance at 0x10422b7a0>,
 <lib.Buses.Point instance at 0x10422b830>,
 <lib.Buses.Point instance at 0x10422b8c0>,
 <lib.Buses.Point instance at 0x10422b950>,
 <lib.Buses.Point instance at 0x10422b9e0>,
 <lib.Buses.Point instance at 0x10422ba70>,
 <lib.Buses.Point instance at 0x10422bb00>,
 <lib.Buses.Point instance at 0x10422bb90>,
 <lib.Buses.Point instance at 0x10422bc20>,
 <lib.Buses.Point instance at 0x10422bcb0>,
 <lib.Buses.Point instance at 0x10422bd40>,
 <lib.Buses.Point instance at 0x10422bdd0>,
 <lib.Buses.Point instance at 0x10422be60>,
 <lib.Buses.Point instance at 0x10422bef0>,
 <lib.Buses.Point instance at 0x10422bf80>,
 <lib.Buses.Point instance at 0x104231050>,
 <lib.Buses.Point instance at 0x1042310e0>,
 <lib.Buses.Point instance at 0x104231170>,
 <lib.Buses.Point instance at 0x104231200>,
 <lib.Buses.Point instance at 0x1

In [22]:
# create waypoints 

# create groups (10 waypoints, max = 24tk)
waypoint_groups=strider(map_points,10)
waypoint_groups

[[<lib.Buses.Stop instance at 0x10422b5f0>,
  <lib.Buses.Point instance at 0x10422bb90>,
  <lib.Buses.Point instance at 0x104231170>,
  <lib.Buses.Point instance at 0x104231710>,
  <lib.Buses.Point instance at 0x104231cb0>,
  <lib.Buses.Point instance at 0x104234290>,
  <lib.Buses.Stop instance at 0x104234830>,
  <lib.Buses.Stop instance at 0x104234dd0>,
  <lib.Buses.Point instance at 0x1042373b0>,
  <lib.Buses.Point instance at 0x104237680>,
  <lib.Buses.Point instance at 0x104237950>,
  <lib.Buses.Point instance at 0x104237c20>,
  <lib.Buses.Stop instance at 0x104237ef0>,
  <lib.Buses.Point instance at 0x10423b200>,
  <lib.Buses.Point instance at 0x10423b4d0>,
  <lib.Buses.Stop instance at 0x10423b7a0>,
  <lib.Buses.Point instance at 0x10423ba70>,
  <lib.Buses.Point instance at 0x10423bd40>,
  <lib.Buses.Stop instance at 0x10423b050>,
  <lib.Buses.Stop instance at 0x104240320>,
  <lib.Buses.Point instance at 0x1042405f0>,
  <lib.Buses.Point instance at 0x1042408c0>,
  <lib.Buses.Stop

In [21]:
# chose first point in each group
waypoints_now = []
for g in waypoint_groups:
  waypoints_now.append(g[0])
waypoints_now

[<lib.Buses.Stop instance at 0x10422b5f0>,
 <lib.Buses.Point instance at 0x10422b680>,
 <lib.Buses.Point instance at 0x10422b710>,
 <lib.Buses.Point instance at 0x10422b7a0>,
 <lib.Buses.Point instance at 0x10422b830>,
 <lib.Buses.Point instance at 0x10422b8c0>,
 <lib.Buses.Point instance at 0x10422b950>,
 <lib.Buses.Point instance at 0x10422b9e0>,
 <lib.Buses.Point instance at 0x10422ba70>,
 <lib.Buses.Point instance at 0x10422bb00>,
 <lib.Buses.Point instance at 0x10422bb90>,
 <lib.Buses.Point instance at 0x10422bc20>,
 <lib.Buses.Point instance at 0x10422bcb0>,
 <lib.Buses.Point instance at 0x10422bd40>,
 <lib.Buses.Point instance at 0x10422bdd0>,
 <lib.Buses.Point instance at 0x10422be60>,
 <lib.Buses.Point instance at 0x10422bef0>,
 <lib.Buses.Point instance at 0x10422bf80>,
 <lib.Buses.Point instance at 0x104231050>,
 <lib.Buses.Point instance at 0x1042310e0>,
 <lib.Buses.Point instance at 0x104231170>,
 <lib.Buses.Point instance at 0x104231200>,
 <lib.Buses.Point instance at 0x1

In [23]:
waypoints_now = []
for g in waypoint_groups:
  waypoints_now.append(g[0])
waypoints_now

[<lib.Buses.Stop instance at 0x10422b5f0>,
 <lib.Buses.Point instance at 0x10422b680>,
 <lib.Buses.Point instance at 0x10422b710>,
 <lib.Buses.Point instance at 0x10422b7a0>,
 <lib.Buses.Point instance at 0x10422b830>,
 <lib.Buses.Point instance at 0x10422b8c0>,
 <lib.Buses.Point instance at 0x10422b950>,
 <lib.Buses.Point instance at 0x10422b9e0>,
 <lib.Buses.Point instance at 0x10422ba70>,
 <lib.Buses.Point instance at 0x10422bb00>]

In [33]:
# build the list of tuples waypoint lat,lon

waypoints_tuples=[]

for wp in waypoints_now:
    x = float(wp.lat)
    y = float(wp.lon)
    waypoints_tuples.append((x,y))

waypoints_tuples

[(40.75636999999988, -73.99075000000028),
 (40.75627572667, -73.990441009325),
 (40.756475650427, -73.990296721458),
 (40.757195698684, -73.991826653218),
 (40.757780833695, -73.993073344263),
 (40.759069737851, -73.99211847769),
 (40.758590261895, -73.991017699373),
 (40.758213180119, -73.990079998839),
 (40.75789542208, -73.98939120776),
 (40.756565860105, -73.990337490905)]

In [34]:
# make the map using the first item from each 
import config
gmaps.configure(api_key=config.paid_directions_api_key)

# get length of the array and use that as the end point
# WHY IS IT STILL DOUBLED!!!!
end_num = len(waypoints_now)-1

start = (float(waypoints_now[0].lat), float(waypoints_now[0].lon))
end = (float(waypoints_now[end_num].lat), float(waypoints_now[end_num].lon))

waypoint_map = gmaps.figure()
# points_layer = gmaps.directions_layer(
#        start, end, waypoints=[waypoints_now[1:-2]])
points_layer = gmaps.directions_layer(
        start, end, waypoints=[x for x in waypoints_tuples])
waypoint_map.add_layer(points_layer)
waypoint_map
                

Figure(layout=FigureLayout(height=u'420px'))